In [ ]:
from ml_tools.ML_datasetmaster import DragonDatasetMulti
from ml_tools.ML_models import DragonTabularTransformer
from ml_tools.ML_models_advanced import DragonNodeModel
from ml_tools.ML_trainer import DragonTrainer
from ml_tools.ML_callbacks import DragonModelCheckpoint, DragonEarlyStopping, DragonLRScheduler
from ml_tools.ML_utilities import build_optimizer_params
from ml_tools.ML_configuration import (
    MultiTargetRegressionMetricsFormat, 
    FinalizeMultiTargetRegression, 
    DragonNodeParams, 
    DragonTabularTransformerParams, 
    DragonTrainingConfig
    )
from ml_tools.ML_utilities import inspect_model_architecture
from ml_tools.utilities import load_dataframe_with_schema
from ml_tools.custom_logger import train_logger
from ml_tools.schema import FeatureSchema

from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from paths import PM
from helpers.constants import TARGETS

## 1. Parameters

In [ ]:
train_config = DragonTrainingConfig(
    validation_size=0.2,
    test_size=0.1,
    initial_learning_rate=0.001,
    batch_size=64,
    random_state=101,
    early_stop_patience=12,
    scheduler_patience=3,
    scheduler_lr_factor=0.5
)

## 2. Load Schema and Dataframe

In [ ]:
schema = FeatureSchema.from_json(PM.optimization_engineering)

df, _ = load_dataframe_with_schema(df_path=PM.optimization_data_file, schema=schema)

## 3. Make Datasets

In [ ]:
dataset = DragonDatasetMulti(pandas_df=df,
                             target_columns=TARGETS,
                             schema=schema,
                             kind="multitarget regression",
                             feature_scaler="fit",
                             target_scaler="fit",
                             validation_size=train_config.validation_size,
                             test_size=train_config.test_size,
                             random_state=train_config.random_state)

## 4. Model and Trainer

In [ ]:
### MODEL SELECTION ###
use_NODE = True
#######################

if use_NODE:
    # NODE
    model_params = DragonNodeParams(
        schema=schema,
        out_targets=dataset.number_of_targets,
        embedding_dim=16,    
        num_trees=512,       
        num_layers=1,        
        tree_depth=5,
        additional_tree_output_dim=2,
        input_dropout=0.1,
        embedding_dropout=0.1,
        choice_function='entmax',
        bin_function='entmoid',
        batch_norm_continuous=False
    )
    
    model = DragonNodeModel(**model_params)
    # Initialize decision thresholds before training.
    model.data_aware_initialization(train_dataset=dataset.train_dataset, num_samples=1000)
    
else:
    # Tabular transformer
    model_params = DragonTabularTransformerParams(
        schema=schema,
        out_targets=dataset.number_of_targets,
        embedding_dim=1024,
        num_heads=4,
        num_layers=2,
        dropout=0.1
    )
    
    model = DragonTabularTransformer(**model_params)

# optimizer
optim_params = build_optimizer_params(model=model, weight_decay=0.001)

optimizer = AdamW(params=optim_params, lr=train_config.initial_learning_rate)

trainer = DragonTrainer(model=model,
                        train_dataset=dataset.train_dataset,
                        validation_dataset=dataset.validation_dataset,
                        kind="multitarget regression",
                        optimizer=optimizer,
                        device="cuda:0",
                        checkpoint_callback=DragonModelCheckpoint(save_dir=PM.optimization_train_checkpoints),
                        early_stopping_callback=DragonEarlyStopping(patience=train_config.early_stop_patience), # type: ignore
                        lr_scheduler_callback=DragonLRScheduler(scheduler=ReduceLROnPlateau(optimizer=optimizer,
                                                                                            factor=train_config.scheduler_lr_factor, # type: ignore
                                                                                            patience=train_config.scheduler_patience)) # type: ignore
                        )

## 5. Training

In [ ]:
history = trainer.fit(save_dir=PM.optimization_train_artifacts,
                    epochs=500,
                    batch_size=train_config.batch_size)

## 6. Evaluation

In [ ]:
trainer.evaluate(save_dir=PM.optimization_train_evaluation,
                 model_checkpoint="latest",
                 test_data=dataset.test_dataset,
                 val_format_configuration=MultiTargetRegressionMetricsFormat(),
                 test_format_configuration=MultiTargetRegressionMetricsFormat(scatter_color='tab:brown'))

## 7. Explanation

In [ ]:
trainer.explain_captum(save_dir=PM.optimization_train_evaluation,
                       n_samples=200,
                       n_steps=100)

## 8. Save artifacts

In [ ]:
# Dataset
dataset.save_artifacts(PM.optimization_train_artifacts)

# Model
model.save(PM.optimization_train_artifacts)
inspect_model_architecture(model=model, save_dir=PM.optimization_train_artifacts)

# Train log
train_log = train_config.to_log() | model_params.to_log() | history

train_logger(data=train_log, save_directory=PM.optimization_train_metrics)

## 9. Finalize Deep Learning

In [ ]:
trainer.finalize_model_training(model_checkpoint='current',
                                save_dir=PM.optimization_train_artifacts,
                                finalize_config=FinalizeMultiTargetRegression(filename="lithium_multitarget_node",
                                                                              target_names=dataset.target_names))